In [25]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from sympy import symbols, sympify

class Candidate:
    
    #누가 참여하는지 + 기본값
    def __init__(self, candidate_name_list, candidate_init_value_list):
        """Initialize the candidate with name and initial values."""
        self.candidate_dict = dict(zip(candidate_name_list, candidate_init_value_list))
        self.polling_event = []

    def parse_polling_formula(self, formula, var1, var2):
        """Parse the polling formula to create a sympy expression."""
        vars = symbols(f'{var1} {var2}')
        try:
            expression = sympify(formula)
            is_valid = True
        except:
            expression = sympify(f"{vars[0]}-{vars[0]}")  # Default expression
            is_valid = False
        return expression, is_valid

    def add_polling_event(self, event_name_list, event_value_list, prize_name, prize_count, formula, var1, var2):
        """Add a polling event and its evaluation."""
        self.polling_event.append(PollingEvent(event_name_list, event_value_list, prize_name, prize_count))
        self.add_polling_evaluation(len(self.polling_event) - 1, formula, var1, var2)


    def get_polling_weight(self, event_index, parsing_formula, var1, var2):
        """Calculate polling weights based on the given formula."""
        event = self.polling_event[event_index]
        expression, is_valid = self.parse_polling_formula(parsing_formula, var1, var2)

        if not is_valid:
            return "Invalid Formula"
        
        polling_weight = {}
        for key in event.event_participants:
            values = {var1: self.candidate_dict.get(key, 0), var2: event.event_participants[key]}
            substituted_expression = expression.subs(values)
            result = substituted_expression.evalf()
            polling_weight[key] = result
        return polling_weight

    def add_polling_evaluation(self, event_index, parsing_formula, var1, var2):
        polling_weight = self.get_polling_weight(event_index, parsing_formula, var1, var2)
        self.polling_event[event_index].get_event_evaluation(polling_weight)
        return self

    def __str__(self):
        """String representation of Candidate object."""
        details = ["Candidate", str(self.candidate_dict), "\nPolling Event"]
        details.extend(str(event) for event in self.polling_event)
        return "\n".join(details)
    
class PollingEvent:
    def __init__(self, event_name_list, event_value_list, prize_name, prize_count):
        print("hi")
        self.event_participants = dict(zip(event_name_list, event_value_list))
        self.prize_name = prize_name
        self.prize_count = prize_count
        self.event_evaluated = {}
    
    def get_event_evaluation(self, evaluation):
        self.event_evaluated = evaluation
        return self
    
    def __str__(self):
        
        return str(self.event_participants) + " " + self.prize_name + " " + str(self.prize_count) +"\nCalculate: " + str(self.event_evaluated)



In [27]:
#File 형식은 유저-
def file_opener(filename, printme = False):
    candidate_name_list = []
    candidate_init_value_list = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for index, line  in enumerate(lines):
            if index%2 == 0:
                candidate_name_list.append(line.strip())
            else:
                if line.strip() == '':
                    candidate_init_value_list.append(1)
                else:
                    candidate_init_value_list.append(int(line.strip()))
    if len(candidate_name_list) > len(candidate_init_value_list):
        candidate_init_value_list.append(1)
    if printme:
        print("Data obtained from file " + filename+ " :")
        print(candidate_name_list)
        print(candidate_init_value_list)
        print(len(candidate_name_list))
        print(len(candidate_init_value_list))
    return candidate_name_list, candidate_init_value_list

players, gall_activity = file_opener('Players.txt', True)
UCS_writer, UCS_score = file_opener('UCSwriter.txt', True)
UCS_player, UCS_activity = file_opener('UCSplayer.txt', True)

Data obtained from file Players.txt :
['히메사카노아', '만중', '설화', '후후훈', 'srhgim', 'refos', '마야', '124Y01SKY', 'Junare', 'Beez', 'HL5T', 'N진화', 'OTDS', '옥돔피트', '채은서', 'Kafuu', 'Plutz', '무초', '솔2', '승마', 'Asura', '이너임', '이로오라']
[5, 5, 5, 5, 4, 5, 5, 3, 3, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
23
23
Data obtained from file UCSwriter.txt :
['히메사카노아', '만중', '설화', '후후훈', 'srhgim', 'refos', '마야', '124Y01SKY', 'Junare', 'Beez', 'HL5T', 'N진화']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
12
12
Data obtained from file UCSplayer.txt :
['N진화', 'OTDS', '옥돔피트', '채은서', 'Kafuu', 'Plutz', 'Junare', '무초', 'refos', '솔2', '승마', 'Asura', '이너임', '이로오라']
[5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5]
14
14


In [28]:
#candidate들을 dictionary로 저장합니다.
candidates = Candidate(players, gall_activity)
print(candidates)

Candidate
{'히메사카노아': 5, '만중': 5, '설화': 5, '후후훈': 5, 'srhgim': 4, 'refos': 5, '마야': 5, '124Y01SKY': 3, 'Junare': 3, 'Beez': 0, 'HL5T': 0, 'N진화': 0, 'OTDS': 5, '옥돔피트': 5, '채은서': 5, 'Kafuu': 5, 'Plutz': 5, '무초': 5, '솔2': 5, '승마': 5, 'Asura': 5, '이너임': 5, '이로오라': 5}

Polling Event


In [29]:
candidates.add_polling_event(UCS_writer, UCS_score, "Chicken", 2, "min(x,5) * y", "x", "y")
candidates.add_polling_event(UCS_player, UCS_activity, "Hamburger", 7, "max(1, min(x,5)) * y", "x", "y")


hi
hi


In [30]:
print(candidates)

Candidate
{'히메사카노아': 5, '만중': 5, '설화': 5, '후후훈': 5, 'srhgim': 4, 'refos': 5, '마야': 5, '124Y01SKY': 3, 'Junare': 3, 'Beez': 0, 'HL5T': 0, 'N진화': 0, 'OTDS': 5, '옥돔피트': 5, '채은서': 5, 'Kafuu': 5, 'Plutz': 5, '무초': 5, '솔2': 5, '승마': 5, 'Asura': 5, '이너임': 5, '이로오라': 5}

Polling Event
{'히메사카노아': 1, '만중': 1, '설화': 1, '후후훈': 1, 'srhgim': 1, 'refos': 1, '마야': 1, '124Y01SKY': 1, 'Junare': 1, 'Beez': 1, 'HL5T': 1, 'N진화': 1} Chicken 2
Calculate: {'히메사카노아': 5.00000000000000, '만중': 5.00000000000000, '설화': 5.00000000000000, '후후훈': 5.00000000000000, 'srhgim': 4.00000000000000, 'refos': 5.00000000000000, '마야': 5.00000000000000, '124Y01SKY': 3.00000000000000, 'Junare': 3.00000000000000, 'Beez': 0, 'HL5T': 0, 'N진화': 0}
{'N진화': 5, 'OTDS': 5, '옥돔피트': 5, '채은서': 5, 'Kafuu': 1, 'Plutz': 5, 'Junare': 5, '무초': 5, 'refos': 5, '솔2': 5, '승마': 5, 'Asura': 5, '이너임': 5, '이로오라': 5} Hamburger 7
Calculate: {'N진화': 5.00000000000000, 'OTDS': 25.0000000000000, '옥돔피트': 25.0000000000000, '채은서': 25.0000000000000, 'Kafuu': 5.000

In [77]:
import random
import time

class WeightedVote:

    def __init__(self, candidates):
        self.candidates = candidates
        self.total_picked_candidates = []
    def do_weighted_vote(self, type = "One By One", sleep_time = None, duplicate = False):
        if type == "One By One":
            return self.poll_one_event(duplicate)
        elif type == "At Once":
            return self.spontaneous(sleep_time, duplicate)
        else:
            return "Invalid Type"
    
    @staticmethod
    def select_candidate(candidates_weights):
        total_weight = sum(candidates_weights.values())
        probabilities = {candidate: weight / total_weight for candidate, weight in candidates_weights.items()}
        candidates, probs = zip(*probabilities.items())
        selected_candidate = random.choices(candidates, weights=probs, k=1)[0]
        return selected_candidate
    
    def select_one_candidate(self, polling_index): #TODO
        selected_candidate = self.select_candidate(self.candidates.polling_event[polling_index].event_evaluated)
        return selected_candidate

    def poll_one_event(self, polling_index, sleep_time, prevent_duplicate):
        #print("selected: " + str(self.total_picked_candidates))
        candidates = []

        for _ in range(self.candidates.polling_event[polling_index].prize_count):
            selected_candidate = self.select_one_candidate(polling_index)
            while prevent_duplicate == True and selected_candidate in self.total_picked_candidates or selected_candidate in candidates:
                selected_candidate = self.select_one_candidate(polling_index)            
            candidates.append(selected_candidate)
            if prevent_duplicate:
                self.total_picked_candidates.append(selected_candidate)
            #print(f"Selected Candidate: {selected_candidate}")
            time.sleep(sleep_time)
        #print(candidates)
        return candidates
    
    def poll_all_events(self, sleep_time, prevent_duplicate):
        poll_results = []
        for i in range(len(self.candidates.polling_event)):
            one_poll = self.poll_one_event(i, sleep_time, prevent_duplicate)
            poll_results.append(one_poll)
            #print("####")
            #print(poll_results)
        return poll_results
    
    def announce_winner(self, poll_results):
        for i in range(len(poll_results)):
            print(f"Prize: {self.candidates.polling_event[i].prize_name}")
            print(f"Winners: {poll_results[i]}")
        print()
    def __str__(self):
        return str(self.weighted_vote)
    
    def purge(self):
        self.total_picked_candidates = []
        return self

In [78]:
weighted_vote = WeightedVote(candidates)
for i in range(6):
    poll_results = weighted_vote.poll_all_events(0.01, True)
    weighted_vote.announce_winner(poll_results)
    weighted_vote.purge()

Prize: Chicken
Winners: ['만중', '후후훈']
Prize: Hamburger
Winners: ['Plutz', '승마', 'refos', '이너임', '옥돔피트', 'Asura', '솔2']

Prize: Chicken
Winners: ['후후훈', 'srhgim']
Prize: Hamburger
Winners: ['솔2', 'OTDS', 'Asura', '승마', '채은서', 'Junare', '이너임']

Prize: Chicken
Winners: ['만중', 'refos']
Prize: Hamburger
Winners: ['솔2', 'Junare', '옥돔피트', 'OTDS', 'Asura', 'N진화', '이너임']

Prize: Chicken
Winners: ['히메사카노아', '124Y01SKY']
Prize: Hamburger
Winners: ['OTDS', '이너임', 'Plutz', 'refos', 'Asura', '무초', '이로오라']

Prize: Chicken
Winners: ['124Y01SKY', '설화']
Prize: Hamburger
Winners: ['OTDS', 'Asura', '무초', '이너임', 'Junare', '솔2', 'refos']

Prize: Chicken
Winners: ['마야', 'refos']
Prize: Hamburger
Winners: ['무초', '솔2', '승마', 'Asura', '이로오라', '이너임', '옥돔피트']



In [19]:

candidates_weights = {
    'Alice': 1.5,
    'Bob': 2.0,
    'Charlie': 1.0,
}
def select_candidate(candidates_weights):
    total_weight = sum(candidates_weights.values())
    probabilities = {candidate: weight / total_weight for candidate, weight in candidates_weights.items()}
    candidates, probs = zip(*probabilities.items())
    selected_candidate = random.choices(candidates, weights=probs, k=1)[0]
    return selected_candidate


select_candidate(candidates_weights)

'Bob'